## Canned Goods Classification using Transfer Learning

Modified from https://colab.sandbox.google.com/github/tensorflow/examples/blob/master/community/en/flowers_tf_lite.ipynb

In [ ]:
import tensorflow as tf
assert tf.__version__.startswith('2')

import os
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
tf.__version__

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Setup Input Pipeline

In [ ]:
base_dir = '/content/drive/My Drive/cans-final-dataset-16-items'

In [ ]:
for d in os.listdir(base_dir):
  try:
    ims = [f for f in os.listdir(os.path.join(base_dir, d))]
    print(d, len(ims))
  except:
    pass

Use `ImageDataGenerator` to rescale the images.

Create the train generator and specify where the train dataset directory, image size, batch size.

Create the validation generator with similar approach as the train generator with the flow_from_directory() method.

In [ ]:
IMAGE_SIZE = 224
BATCH_SIZE = 64

datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255, 
    validation_split=0.2)

train_generator = datagen.flow_from_directory(
    base_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE, 
    subset='training')

val_generator = datagen.flow_from_directory(
    base_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE, 
    subset='validation')

In [ ]:
for image_batch, label_batch in train_generator:
  break
image_batch.shape, label_batch.shape

Save the labels in a file which will be downloaded later.

In [ ]:
print (train_generator.class_indices)

labels = '\n'.join(sorted(train_generator.class_indices.keys()))
labels = labels.replace('-', ' ')
labels = labels.replace('_', ' ')

with open('labels.txt', 'w') as f:
  f.write(labels)

In [ ]:
!cat labels.txt

## Create the base model from the pre-trained convnets

Create the base model from the **MobileNet V2** model developed at Google, and pre-trained on the ImageNet dataset, a large dataset of 1.4M images and 1000 classes of web images.

Load the MobileNetV2 from TensorFlow. Since we are using the pretrained model only for feature extraction and will add and train a classification layer specific to our canned goods dataset, we add the argument `include_top=False` to remove the ImageNet classification layer.

In [ ]:
IMG_SHAPE = (IMAGE_SIZE, IMAGE_SIZE, 3)

# Create the base model from the pre-trained model MobileNet V2
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                              include_top=False, 
                                              weights='imagenet')

## Feature extraction
Freeze the convolutional base created from the previous step, using it as a feature extractor. Add a classifier on top of the base model and train the top-level classifier.

In [ ]:
base_model.trainable = False

### Add a classification head

In [ ]:
model = tf.keras.Sequential([
  base_model,
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.GlobalAveragePooling2D(),
  tf.keras.layers.Dense(16, activation='softmax')
])

### Compile the model

You must compile the model before training it.  Since there are multiple classes, use a categorical cross-entropy loss.

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
print('Number of trainable variables = {}'.format(len(model.trainable_variables)))

### Train the model

<!-- TODO(markdaoust): delete steps_per_epoch in TensorFlow r1.14/r2.0 -->

In [ ]:
epochs = 40

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=4)

history = model.fit(train_generator, 
                    steps_per_epoch=len(train_generator), 
                    epochs=epochs, 
                    validation_data=val_generator, 
                    validation_steps=len(val_generator),
                    callbacks = [callback])

### Learning curves

Let's take a look at the learning curves of the training and validation accuracy/loss when using the MobileNet V2 base model as a fixed feature extractor. 

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

### Evaluate the Model

In [ ]:
import cv2
from google.colab.patches import cv2_imshow
import random

In [ ]:
# get a batch of validation images
for image_batch, label_batch in val_generator:
  break
image_batch.shape, label_batch.shape

In [ ]:
model.evaluate (image_batch, label_batch)

In [ ]:
label_dct = {}
label_lst = labels.split('\n')
for i in range(len(label_lst)):
  label_dct[i] = label_lst[i]
label_dct


In [ ]:
pred_arr = model.predict(image_batch)
preds = [np.argmax(a) for a in pred_arr]
y_true = [np.argmax(a) for a in label_batch]
incorrect = 0
total = 0
for p, y, img in zip(preds, y_true, image_batch):
#  print(p,y)
  total += 1
  if p != y:
    print("Predicted Class:", label_dct[p])
    print("Actual Class:", label_dct[y])
#    print(label_dct[p])
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    cv2_imshow(img*255)
    incorrect += 1

The images above show that the model has some difficulty with fine-grained differences. For example, it classified all the Campbell's Tomato Soup as Campbell's Chicken Noodle Soup. These cans are nearly identical except for the words "Tomato" and "Chicken Noodle". Similarly, the model has some trouble with the three kinds of diced tomatoes from WinCo.

In [ ]:
import pandas as pd
pd.DataFrame(tf.math.confusion_matrix(y_true, preds).numpy())

In [ ]:
print("Number incorrect:", incorrect)
print("Total items: ", total)
print("Accuracy: ", str((total-incorrect)*100/total) + '%')

## Convert to TFLite

Saved the model using `tf.saved_model.save` and then convert the saved model to a tf lite compatible format.

In [ ]:
saved_model_dir = 'save/cans_model'
tf.saved_model.save(model, saved_model_dir)

converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
tflite_model = converter.convert()

with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

Download the converted model and labels

In [ ]:
from google.colab import files

files.download('model.tflite')
files.download('labels.txt')